In [1]:
#Preparing The Environment
import quandl
df = quandl.get("CHRIS/CME_ES2")
df.head()
df.tail()
df.shape

(5789, 8)

In [2]:
#Creating the predictors
import talib as ta
df['EMA10'] = ta.EMA(df['Settle'].values, timeperiod=10)
df['EMA30'] = ta.EMA(df['Settle'].values, timeperiod=30)
df['ATR'] = ta.ATR(df['High'].values, df['Low'].values, df['Settle'].values, timeperiod=14)
df['ADX'] = ta.ADX(df['High'].values, df['Low'].values, df['Settle'].values, timeperiod=14)
df['RSI'] = ta.RSI(df['Settle'].values, timeperiod=14)
macd, macdsignal, macdhist = ta.MACD(df['Settle'].values, fastperiod=12, slowperiod=26, signalperiod=9)
df['MACD'] = macd
df['MACDsignal'] = macdsignal
df.tail()

ModuleNotFoundError: No module named 'talib'

In [1]:
pip install talib

  Using cached talib-0.1.1.tar.gz (1.3 kB)
  ERROR: Command errored out with exit status 1:
   command: /Users/miaoyuesun/anaconda3/envs/py36/bin/python -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'/private/var/folders/7_/3f3b7cs95t53f839zr3wfhqw0000gn/T/pip-install-45lrit8h/talib/setup.py'"'"'; __file__='"'"'/private/var/folders/7_/3f3b7cs95t53f839zr3wfhqw0000gn/T/pip-install-45lrit8h/talib/setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d /private/var/folders/7_/3f3b7cs95t53f839zr3wfhqw0000gn/T/pip-wheel-w9q8fdh9
       cwd: /private/var/folders/7_/3f3b7cs95t53f839zr3wfhqw0000gn/T/pip-install-45lrit8h/talib/
  Complete output (29 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib
  creating build/lib/talib
  copying talib/__init__.py -> build/lib/talib
  copying talib/cli.py -> bu

In [ ]:
import numpy as np
df['ClgtEMA10'] = np.where(df['Settle'] > df['EMA10'], 1, -1)
df['EMA10gtEMA30'] = np.where(df['EMA10'] > df['EMA30'], 1, -1)
df['MACDSIGgtMACD'] = np.where(df['MACDsignal'] > df['MACD'], 1, -1)
df.tail()

In [ ]:
#Creating the target variables
df['Return'] = df['Settle'].pct_change(1).shift(-1)
df['target_cls'] = np.where(df.Return > 0, 1, 0)
df['target_rgs'] = df['Return']
df.tail()

In [ ]:
#Obtaining the data set for decision trees
predictors_list = ['ATR', 'ADX','RSI', 'ClgtEMA10', 'EMA10gtEMA30', 'MACDSIGgtMACD']
X = df[predictors_list]
X.tail()

In [ ]:
y_cls = df.target_cls
y_cls.tail()

In [ ]:
y_rgs = df.target_rgs
y_rgs.tail()

In [ ]:
#Splitting the data into training and testing data sets
from sklearn.model_selection import train_test_split
y=y_cls
X_cls_train, X_cls_test, y_cls_train, y_cls_test = train_test_split(X, y, test_size=0.3, random_state=432, stratify=y)

print (X_cls_train.shape, y_cls_train.shape)
print (X_cls_test.shape, y_cls_test.shape)

In [ ]:
train_length = int(len(df)*0.70)
X_rgs_train = X[:train_length]
X_rgs_test = X[train_length:]
y_rgs_train = y_rgs[:train_length]
y_rgs_test = y_rgs[train_length:]

print (X_rgs_train.shape, y_rgs_train.shape)
print (X_rgs_test.shape, y_rgs_test.shape)

In [ ]:
#Decision Trees for Classification
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(criterion='gini', max_depth=3, min_samples_leaf=6)
clf

In [ ]:
import pandas as pd
from sklearn.impute import SimpleImputer
X_cls_train_drop = pd.DataFrame(SimpleImputer().fit_transform(X_cls_train))
clf = clf.fit(X_cls_train_drop, y_cls_train)
clf

In [ ]:
#Visualize Decision Trees for Classification
from sklearn import tree
import graphviz
dot_data = tree.export_graphviz(clf, out_file=None,filled=True,feature_names=predictors_list)
graphviz.Source(dot_data)

In [ ]:
#Make forecast
X_cls_test_imputed = pd.DataFrame(SimpleImputer().fit_transform(X_cls_test))
y_cls_pred = clf.predict(X_cls_test_imputed)

In [ ]:
#Performance analysis
from sklearn.metrics import classification_report
report = classification_report(y_cls_test, y_cls_pred)
print(report)

In [ ]:
# Regression tree model
from sklearn.tree import DecisionTreeRegressor
dtr = DecisionTreeRegressor(min_samples_leaf = 200)

In [ ]:
X_rgs_train_imputed = pd.DataFrame(SimpleImputer().fit_transform(X_rgs_train))
dtr.fit(X_rgs_train_imputed, y_rgs_train)


In [ ]:
#Visualize the model
from sklearn import tree
import graphviz
dot_data = tree.export_graphviz(dtr,
                  out_file=None,
                  filled=True,
                  feature_names=predictors_list)
graphviz.Source(dot_data)